# Sentiment analysis 

First parse and read in the file.  Each line is of the form:


0:	topic category label (books,	camera,	dvd,	health,	music,	or	software)	
1:	sentiment	category	label	(pos	or	neg)	
2:	document	identifier	
3	and	on:	the	document tokens

We are only interested in 1 and 3.  Note that once a line is 'split' number 3 (above), the sentence is split into individual words.  These need to be 'joined' to form a sentence.


In [1]:
def read_file(fname='all_sentiment_shuffled.txt'):
    fp = open(fname, 'r')
    all_labels = []
    all_text = []
    for line in fp:
        
        # TODO
        
    return( all_labels, all_text )

As with the digits exercise, we return a list of labels and items to be classified in this case the sentences commenting on the product

In [2]:
all_labels, all_text = read_file()

The below will only work on MacOS.  'wc'  (word count) is a unix program that counts the number of characters, words, and lines in a file.  With the command line argument is just counts the number of lines.  Note the '!' at the beginning of the file which tells the notebook that what follows is a shell command.  Note sure if these is a windows equivalent; windows users may have to comment this out. You can also check to see the number of entries by openign the file in an editor.

In [3]:
!wc -l 'all_sentiment_shuffled.txt'

   11914 all_sentiment_shuffled.txt


*Sanity check:*  have we read and processed the whole file: number of labels and sentences should equal the number of lines in the file.

In [4]:
(len(all_labels), len(all_text))

(11914, 11914)

Check a few elements.  e.g., look at line 1

In [5]:
(all_labels[0], all_text[0])

('neg',
 "i bought this album because i loved the title song . it 's such a great song , how bad can the rest of the album be , right ? well , the rest of the songs are just filler and are n't worth the money i paid for this . it 's either shameless bubblegum or oversentimentalized depressing tripe . kenny chesney is a popular artist and as a result he is in the cookie cutter category of the nashville music scene . he 's gotta pump out the albums so the record company can keep lining their pockets while the suckers out there keep buying this garbage to perpetuate more garbage coming out of that town . i 'll get down off my soapbox now . but country music really needs to get back to it 's roots and stop this pop nonsense . what country music really is and what it is considered to be by mainstream are two different things .")

## Bag of Words model

The below URL that describes how to use `CountVectorizer`.  

http://scikit-learn.org/stable/modules/feature_extraction.html

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [21]:
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And the third one.',
     'Is this the first document?',
 ]

In [25]:
# how to use vectorizer?




['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

In [26]:
X.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

In [27]:
corpus[1]

'This is the second second document.'

As described in the documentation at the URL, pass in all the text --- note that 'all_text' is an array of sentences, not an array of arrays of words.  When reading in the file the tokens should have been 'joined' to recreate the sentence.

In [7]:
# We need to produce X_train
# how



A check to see how big the training matrix is.  It should have one entry for each sentence read in (11914).  The total number of unique words in the corpus turns out to be 46925

In [8]:
X_train.shape

(11914, 46925)

### How many words (just words) are in the data set?

If one wants, one can look at some of these features just to see what is going on.  Note that the vectorizer stores a sorted version of the features.  Hence the beginning of the feature list is a bunch of numbers.  It is only towards the middle that we see actual words. This is also discussed at the above URL.

In [9]:
f = vectorizer.get_feature_names()
print(len(f))
print(f[:10])
print(f[2000:2010])
print(f[30000:30010])

46925
['00', '000', '0003', '000mb', '004144', '007', '00am', '00pm', '01', '02']
['agendas', 'agent', 'agents', 'agentz', 'ager', 'agers', 'ages', 'agey', 'aggh', 'agglomerations']
['overpriced', 'overproduced', 'overproduction', 'overpronnouncing', 'overprotecting', 'overrated', 'overrating', 'overreach', 'overreached', 'overresponsible']


In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

Feature vectors in this case indicate the count of words --- not real numbers.  Hence GausianNB should NOT be used. By our discussion in class, MultinomialNB should be used as we are dealing with discrete tokens.  BernoulliNB could also be used but we get better performance with Multinomial

In [31]:
model = MultinomialNB()
#model = BernoulliNB()

In [32]:
model.fit(X_train, all_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Another sanity check, we predict on the same set we training on.  Performance should be very high

In [33]:
# expected = ?
# predicted = ?

Did we get the correct number of predictions

In [34]:
len(predicted)

11914

As one would expect, performance should be high --- we are testing on the training set!  If it isn't, then there is something not right in the above steps.

In [15]:
print(metrics.accuracy_score(expected, predicted))

0.919422528118


Now, lets do the 10 fold cross validation

In [16]:
from sklearn.model_selection import cross_val_score
y = all_labels
scores = cross_val_score(model, X_train, y, cv=10, scoring='accuracy')

X_train and y are split into 10 folds.  `cross_val_score` automatically builds training sets with 9 of these folds and tests it against the remaining fold

In [17]:
scores

array([ 0.8045302 ,  0.80788591,  0.83892617,  0.81291946,  0.80940386,
        0.81780017,  0.80268682,  0.79848866,  0.81612091,  0.81024349])

Taking the mean of the score ...

In [18]:
scores.mean()

0.81190056576448644

... we get pretty good performance

## Extensions

Now think of ways of increasing performance

In [20]:
scores = cross_val_score(model, X_train, y, cv=10, scoring='accuracy')